In [3]:
# Importing libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
import os
import nilearn as nil
from nilearn import *
import nibabel as nib
from scipy import ndimage
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from sklearn.metrics import RocCurveDisplay, confusion_matrix
import matplotlib.pyplot as plt

# Extracting and processing image data

- Form 3D matrix structure
- List of images

In [4]:
MRIPathPD = './MRI/PPMI_PD'

MRIsPD = []

for root, dirs, files in os.walk(MRIPathPD):
    
    for file in files:
        
        scanPath = os.path.join(root, file)
        rawScan = nil.image.load_img(scanPath)
        
        niftiFile = nil.image.get_data(rawScan)
        
        depthFactor = 64 / niftiFile.shape[0]
        widthFactor = 128 / niftiFile.shape[1]
        heightFactor = 128 / niftiFile.shape[2]
        
        PDScan = ndimage.zoom(niftiFile, (depthFactor, widthFactor, heightFactor, 1), order = 1)
        
        meanVal = np.mean(PDScan)
        stdVal = np.std(PDScan)
        
        normNifti = (PDScan - meanVal) / (stdVal)
        
        MRIsPD.append(normNifti)

PDLabels = np.ones(len(MRIsPD))

In [6]:
MRIPathHealth = './MRI/PPMI_Healthy'

MRIsHealth = []

for root, dirs, files in os.walk(MRIPathHealth):
    
    for file in files:
        
        scanPath = os.path.join(root, file)
        rawScan = nil.image.load_img(scanPath)
        
        niftiFile = nil.image.get_data(rawScan)
        
        depthFactor = 64 / niftiFile.shape[0]
        widthFactor = 128 / niftiFile.shape[1]
        heightFactor = 128 / niftiFile.shape[2]
        
        healthyScan = ndimage.zoom(niftiFile, (depthFactor, widthFactor, heightFactor, 1), order = 1)
        
        meanVal = np.mean(healthyScan)
        stdVal = np.std(healthyScan)
        
        normNifti = (healthyScan - meanVal) / (stdVal)
        
        MRIsHealth.append(normNifti)
        
healthLabels = np.zeros(len(MRIsHealth))

In [11]:
MRIsPD = np.asarray(MRIsPD, dtype = object)
MRIsHealth = np.asarray(MRIsHealth, dtype = object)

MRIsPD = tf.convert_to_tensor(MRIsPD, dtype = tf.float32)
MRIsHealth = tf.convert_to_tensor(MRIsHealth, dtype = tf.float32)

In [12]:
MRIsTrain = tf.concat(values = [MRIsPD[0:18], MRIsHealth[0:18]], axis = 0)
MRIsTest = tf.concat(values = [MRIsPD[18:], MRIsHealth[18:]], axis = 0)

labelsTrain = np.concatenate((PDLabels[0:18], healthLabels[0:18]))
labelsTest = np.concatenate((PDLabels[18:], healthLabels[18:]))

labelsTrain = tf.convert_to_tensor(labelsTrain)
labelsTest = tf.convert_to_tensor(labelsTest)

In [20]:
inputs = tf.keras.Input((64, 128, 128, 1))

x = layers.Conv3D(filters=64, kernel_size=3, activation='relu')(inputs)
x = layers.BatchNormalization()(x)
x = layers.Conv3D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.MaxPool3D(pool_size=2)(x)
x = layers.BatchNormalization()(x)

x = layers.Conv3D(filters=128, kernel_size=3, activation='relu')(inputs)
x = layers.BatchNormalization()(x)
x = layers.Conv3D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.MaxPool3D(pool_size=2)(x)
x = layers.BatchNormalization()(x)

x = layers.GlobalAveragePooling3D()(x)

x = layers.Dense(units=100, activation='relu')(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(units=10, activation='relu')(x)
x = layers.Dropout(0.3)(x)

outputs = layers.Dense(units=2, activation='relu')(x)

# Define the model.
CNN3D = tf.keras.Model(inputs, outputs, name='CNN_3D')

In [21]:
CNN3D.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

training = CNN3D.fit(MRIsTrain, labelsTrain, epochs = 10)

Epoch 1/10


ResourceExhaustedError: Graph execution error:

Detected at node 'CNN_3D/conv3d_10/Conv3D' defined at (most recent call last):
    File "C:\Users\jonma\anaconda3\envs\parkinsons_model\lib\runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "C:\Users\jonma\anaconda3\envs\parkinsons_model\lib\runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "C:\Users\jonma\anaconda3\envs\parkinsons_model\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\jonma\anaconda3\envs\parkinsons_model\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\jonma\anaconda3\envs\parkinsons_model\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\jonma\anaconda3\envs\parkinsons_model\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()


In [ ]:
loss, accuracy = CNN.evaluate(MRIsTest, labelsTest, verbose = 2)
predictionProbs = CNN.predict(MRIsTest, verbose = 2)    # For specificity and sensitivity calculations

In [ ]:
predictionLabels = predictionProbs.argmax(axis = -1)    # Get prediction classes from probabilities

tn, fp, fn, tp = confusion_matrix(labelsTest, predictionLabels).ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

RocCurveDisplay.from_predictions(labelsTest, predictionLabels)
plt.savefig('ROC_Curve.png')

In [ ]:
print("Model Test Accuracy:", accuracy)
print("Model Test Sensitivity:", sensitivity)
print("Model Test Specificity:", specificity)

metrics = ['Model Test Accuracy: '+str(accuracy), 'Model Test Sensitivity: '+str(sensitivity), 'Model Test Specificity: '+str(specificity)]
with open('3D_CNN_Metrics.txt', 'w') as file:
    file.writelines(metrics)